In [1]:
from imputationLibrary import forwardFilling, randomSampleImputation, meanImputation, movingAverage, splineInterpolation, separate
from imputationLibrary.util import util
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
INPUT, OUTPUT = 'output/\CompleteWeeklyIndexes', 'output/\KNNWeeklyImputed'

In [3]:
df = pd.read_csv(INPUT+'.csv', parse_dates=True, index_col=0)
df_test = df.loc['2015-01-02':] #Selecting test data
df = df.loc['2000-01-01':'2015-01-01'] #Selecting training data

In [4]:
df_test.tail()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n225_Close,n225_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2020-02-14,3380.159912,3.398040e+09,13.680000,0.0,29398.080078,231000000.0,9623.580078,2.222240e+09,23687.589844,69100.0,...,27815.599609,1.445336e+09,1174.349976,204479200.0,NaN,1.23,NaN,NaN,NaN,NaN
2020-02-21,3337.750000,3.899270e+09,17.080000,0.0,28992.410156,311210000.0,9446.690430,2.743010e+09,23386.740234,62100.0,...,27308.810547,1.773613e+09,1164.170044,238290800.0,NaN,6.68,NaN,NaN,NaN,NaN
2020-02-28,2954.219971,8.563850e+09,40.110001,0.0,25409.359375,915990000.0,8461.830078,5.301170e+09,21142.960938,148500.0,...,26129.929688,3.191426e+09,1021.979980,655553200.0,24.443546,40.41,NaN,NaN,NaN,NaN
2020-03-06,2972.370117,6.552140e+09,41.939999,0.0,25864.779297,599780000.0,8530.339844,4.279850e+09,20749.750000,111700.0,...,26146.669922,2.646519e+09,994.770020,465581400.0,NaN,25.19,NaN,NaN,NaN,NaN
2020-03-13,2711.020020,8.258670e+09,57.830002,0.0,23185.619141,843080000.0,7995.259766,4.685890e+09,17431.050781,233400.0,...,24032.910156,5.495156e+09,800.750000,797087400.0,NaN,50.91,NaN,NaN,NaN,NaN


In [5]:
df.head()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n225_Close,n225_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2000-01-07,1441.469971,1.225200e+09,21.719999,0.0,11522.559570,184900000.0,3529.600098,1.634930e+09,18193.410156,0.0,...,15405.629883,0.0,943.880005,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-14,1465.150024,1.085900e+09,19.660000,0.0,11722.980469,266830000.0,3704.739990,1.656630e+09,18956.550781,0.0,...,15542.230469,0.0,973.859985,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-21,1441.359985,1.209800e+09,20.820000,0.0,11251.709961,205840000.0,3849.959961,1.923680e+09,18878.089844,0.0,...,15108.410156,0.0,951.510010,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-28,1360.160034,1.095800e+09,26.139999,0.0,10738.870117,183090000.0,3446.129883,1.616370e+09,19434.779297,0.0,...,16185.940430,0.0,948.210022,0.0,24.412899,0.00,NaN,71.914247,117.577146,0.033573
2000-02-04,1424.369995,1.045100e+09,21.540001,0.0,10963.799805,166590000.0,3874.370117,1.751450e+09,19763.130859,0.0,...,NaN,NaN,1010.429993,0.0,NaN,0.75,NaN,NaN,NaN,NaN


In [6]:
df_white_noise, df_seasonal, df_trended, df_trend_and_seasonal = separate.separate(df)

C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\sklearn\preprocessing\_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\sklearn\preprocessing\_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\sklearn\preprocessing\_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\silva\AppData\Roami

In [7]:
df_white_noise

,vix_Volume,infectious_daily_infect_emv_index,GPR
2000-01-07,0.0,0.00,NaN
2000-01-14,0.0,0.00,NaN
2000-01-21,0.0,0.00,NaN
2000-01-28,0.0,0.00,NaN
2000-02-04,0.0,0.75,NaN
2000-02-11,0.0,0.74,NaN
2000-02-18,0.0,0.00,NaN
2000-02-25,0.0,0.00,NaN
2000-03-03,0.0,0.00,NaN
2000-03-10,0.0,0.00,NaN


In [8]:
df_seasonal

,trade_Trade Policy EMV Fraction
2000-01-07,NaN
2000-01-14,NaN
2000-01-21,NaN
2000-01-28,0.033573
2000-02-04,NaN
2000-02-11,NaN
2000-02-18,NaN
2000-02-25,NaN
2000-03-03,NaN
2000-03-10,NaN


In [9]:
df_trended

,sp500_Volume,Overall EMV Tracker
2000-01-07,1.225200e+09,NaN
2000-01-14,1.085900e+09,NaN
2000-01-21,1.209800e+09,NaN
2000-01-28,1.095800e+09,24.412899
2000-02-04,1.045100e+09,NaN
2000-02-11,1.025700e+09,NaN
2000-02-18,1.042300e+09,NaN
2000-02-25,1.065200e+09,NaN
2000-03-03,1.150300e+09,NaN
2000-03-10,1.138800e+09,NaN


In [10]:
df_trend_and_seasonal

,sp500_Close,vix_Close,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n225_Close,n225_Volume,ftse_Close,ftse_Volume,hsi_Close,hsi_Volume,n100_Close,n100_Volume,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty
2000-01-07,1441.469971,21.719999,11522.559570,184900000.0,3529.600098,1.634930e+09,18193.410156,0.0,6504.799805,8.883060e+08,15405.629883,0.000000e+00,943.880005,0.0,NaN,NaN
2000-01-14,1465.150024,19.660000,11722.980469,266830000.0,3704.739990,1.656630e+09,18956.550781,0.0,6658.200195,1.098626e+09,15542.230469,0.000000e+00,973.859985,0.0,NaN,NaN
2000-01-21,1441.359985,20.820000,11251.709961,205840000.0,3849.959961,1.923680e+09,18878.089844,0.0,6346.299805,1.007934e+09,15108.410156,0.000000e+00,951.510010,0.0,NaN,NaN
2000-01-28,1360.160034,26.139999,10738.870117,183090000.0,3446.129883,1.616370e+09,19434.779297,0.0,6375.600098,9.182270e+08,16185.940430,0.000000e+00,948.210022,0.0,71.914247,117.577146
2000-02-04,1424.369995,21.540001,10963.799805,166590000.0,3874.370117,1.751450e+09,19763.130859,0.0,6185.000000,2.309287e+09,NaN,NaN,1010.429993,0.0,NaN,NaN
2000-02-11,1387.119995,24.420000,10425.209961,193540000.0,3968.889893,1.738590e+09,NaN,NaN,6193.299805,3.433671e+09,17380.300781,0.000000e+00,1014.169983,0.0,NaN,NaN
2000-02-18,1346.089966,26.000000,10219.519531,208930000.0,3965.750000,1.898410e+09,19789.029297,0.0,6165.000000,1.098726e+09,16599.160156,0.000000e+00,992.530029,0.0,NaN,NaN
2000-02-25,1333.359985,25.200001,9862.120117,195240000.0,4178.580078,1.825500e+09,19817.880859,0.0,6198.000000,1.181230e+09,17200.980469,0.000000e+00,1004.719971,0.0,NaN,NaN
2000-03-03,1409.170044,19.209999,10367.200195,210460000.0,4442.870117,2.136530e+09,19927.539063,0.0,6487.500000,1.107792e+09,17285.240234,0.000000e+00,1042.400024,0.0,NaN,NaN
2000-03-10,1395.069946,21.240000,9928.820313,197440000.0,4587.160156,1.992170e+09,19750.400391,0.0,6568.700195,1.005534e+09,17831.859375,0.000000e+00,1045.270020,0.0,NaN,NaN


In [ ]:
### Na biblioteca, criar uma variavel temp pra fazer as transformações, mas na hora de adicionar no df paticionado adicionar a variavel original